In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%cd ..

userData = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
nonBorrower = pd.read_csv('non_borrower_user.csv')
loan = pd.read_csv('loan_activities.csv')

c:\Users\Adli\Desktop\ilegil\bintangkecilmama\DATATHON


**KESIMPULAN**
- test ada yang minjem ada yang ga pernah minjem
- test ada yang direference dan ada yang ga pernah direference
- user yang direference oleh fraud belum tentu fraud
- user yang mereference fraud belum tentu fraud
- Orang yang ga minjem belum tentu non fraud

In [51]:
trainLoan = userData.set_index('user_id').join(loan.set_index('user_id')).reset_index() #nyatuin data train sama loan
trainLoan = trainLoan.drop('pc8', axis=1) #drop kolom yang sama menurut heatmap
trainLoan = trainLoan.drop('pc9', axis=1) #drop kolom yang sama menurut heatmap
userData['-1_count'] = userData.isin([-1]).sum(axis=1) #nambahin kolom missing values (-1)
userData['is_loanreference'] = userData['user_id'].isin(loan["reference_contact"]) #nambahin kolom apakah usernya dijadiin loan reference
userData["has_borrowed"] = userData["user_id"].isin(loan["user_id"]) #nambahin kolom apakah usernya pernah minjem

nonLoan = nonBorrower.set_index('user_id').join(loan.set_index('user_id')).reset_index()

loan["referenced_is_non"] = loan["reference_contact"].isin(nonBorrower["user_id"])
loan["referenced_is_fraud"] = loan["reference_contact"].isin(userData.loc[userData["label"] == 1]["user_id"])

In [3]:
loanFraud = trainLoan.loc[trainLoan["label"]==1]
loanFraud = loanFraud[["user_id", "reference_contact", "loan_type", "ts"]]
loanNonFraud = trainLoan.loc[trainLoan["label"]==0]
loanNonFraud = loanNonFraud[["user_id", "reference_contact", "loan_type", "ts"]]

In [68]:
referenceInfo = pd.DataFrame()
referenceInfo["user_id"] = userData["user_id"].append(nonBorrower["user_id"])
referenceInfo["referenced_by_fraud"] = referenceInfo["user_id"].isin(loanFraud["reference_contact"])
referenceInfo["referenced_by_non"] = referenceInfo["user_id"].isin(loan.loc[loan.user_id.isin(nonBorrower.user_id)]["reference_contact"])
referenceInfo["referencing_fraud"] = referenceInfo["user_id"].isin(loan.loc[loan["referenced_is_fraud"] == 1]["user_id"])
referenceInfo["referencing_non"] = referenceInfo["user_id"].isin(loan.loc[loan["referenced_is_non"] == 1]["user_id"])

C:\Users\Adli\AppData\Local\Temp\ipykernel_15448\2259025806.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  referenceInfo["user_id"] = userData["user_id"].append(nonBorrower["user_id"])


ValueError: Can only compare identically-labeled Series objects